# Data Question 2


## How is queue time impacted by RAM requested? How is it impacted by wall time requested?

## Are there any nodes that fail more than the other ones?


## Identify problematic users
Those people who submit greater than 500 jobs, with each less than 5 minutes within 4 hours

In [1]:
#import libraries 
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from datetime import datetime
import numpy as np

%matplotlib inline

In [2]:
accre_df = pd.read_csv('Data/july-2017-thru-june-2018.txt', sep = '|')
accre_df.head()

/Users/andrewmarsee/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


JobID  JobIDRaw Cluster   Partition      Account        Group    GID  \
0  15806649  15806649   accre  production      h_vuiis      h_vuiis  36052   
1  15806664  15806664   accre  production      h_vuiis      h_vuiis  36052   
2  15806670  15806670   accre  production      h_vuiis      h_vuiis  36052   
3  15806671  15806671   accre  production      h_vuiis      h_vuiis  36052   
4  15811617  15811617   accre  production  cms_samtest  cms_samtest  59297   

         User  UID               Submit  \
0  masispider  NaN  2017-07-02T00:40:58   
1  masispider  NaN  2017-07-02T00:40:58   
2  masispider  NaN  2017-07-02T00:40:58   
3  masispider  NaN  2017-07-02T00:40:58   
4    uscms010  NaN  2017-07-01T05:00:00   

                         ...                              State NNodes NCPUS  \
0                        ...                          COMPLETED      1     2   
1                        ...                          COMPLETED      1     2   
2                        ...                          COMPLETED      1     2   
3                        ...                          COMPLETED      1     2   
4                        ...                          COMPLETED      1     1   

  ReqCPUS  ReqMem ReqGRES                 ReqTRES   Timelimit  NodeList  \
0       2    12Gn     NaN    cpu=2,mem=12G,node=1  9-04:00:00   vmp1085   
1       2    12Gn     NaN    cpu=2,mem=12G,node=1  9-04:00:00   vmp1099   
2       2    12Gn     NaN    cpu=2,mem=12G,node=1  9-04:00:00   vmp1126   
3       2    12Gn     NaN    cpu=2,mem=12G,node=1  9-04:00:00    vmp230   
4       1  4000Mn     NaN  cpu=1,mem=4000M,node=1  2-00:00:00   vmp1242   

                                             JobName  
0  WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...  
1  WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...  
2  WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...  
3  WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...  
4                                    bl_aa7c22e04f21  

[5 rows x 25 columns]

In [3]:
accre_df.head(10)

JobID  JobIDRaw Cluster   Partition      Account        Group    GID  \
0  15806649  15806649   accre  production      h_vuiis      h_vuiis  36052   
1  15806664  15806664   accre  production      h_vuiis      h_vuiis  36052   
2  15806670  15806670   accre  production      h_vuiis      h_vuiis  36052   
3  15806671  15806671   accre  production      h_vuiis      h_vuiis  36052   
4  15811617  15811617   accre  production  cms_samtest  cms_samtest  59297   
5  15811618  15811618   accre  production   cms_stage2   cms_stage1  59298   
6  15811619  15811619   accre  production   cms_stage2          cms  31000   
7  15811621  15811621   accre  production  cms_samtest  cms_samtest  59297   
8  15811624  15811624   accre  production   cms_stage2   cms_stage1  59298   
9  15811625  15811625   accre  production   cms_stage2          cms  31000   

         User       UID               Submit  \
0  masispider       NaN  2017-07-02T00:40:58   
1  masispider       NaN  2017-07-02T00:40:58   
2  masispider       NaN  2017-07-02T00:40:58   
3  masispider       NaN  2017-07-02T00:40:58   
4    uscms010       NaN  2017-07-01T05:00:00   
5     autocms    9209.0  2017-07-01T05:00:01   
6        tuos  126986.0  2017-07-01T05:00:02   
7    uscms010       NaN  2017-07-01T05:00:22   
8     autocms    9209.0  2017-07-01T05:01:01   
9        tuos  126986.0  2017-07-01T05:01:02   

                         ...                              State NNodes NCPUS  \
0                        ...                          COMPLETED      1     2   
1                        ...                          COMPLETED      1     2   
2                        ...                          COMPLETED      1     2   
3                        ...                          COMPLETED      1     2   
4                        ...                          COMPLETED      1     1   
5                        ...                          COMPLETED      1     1   
6                        ...                          COMPLETED      1     1   
7                        ...                          COMPLETED      1     1   
8                        ...                          COMPLETED      1     1   
9                        ...                          COMPLETED      1     1   

  ReqCPUS  ReqMem ReqGRES                 ReqTRES   Timelimit  NodeList  \
0       2    12Gn     NaN    cpu=2,mem=12G,node=1  9-04:00:00   vmp1085   
1       2    12Gn     NaN    cpu=2,mem=12G,node=1  9-04:00:00   vmp1099   
2       2    12Gn     NaN    cpu=2,mem=12G,node=1  9-04:00:00   vmp1126   
3       2    12Gn     NaN    cpu=2,mem=12G,node=1  9-04:00:00    vmp230   
4       1  4000Mn     NaN  cpu=1,mem=4000M,node=1  2-00:00:00   vmp1242   
5       1     2Gc     NaN     cpu=1,mem=2G,node=1    12:00:00   vmp1063   
6       1     2Gc     NaN     cpu=1,mem=2G,node=1    12:00:00   vmp1062   
7       1  4000Mn     NaN  cpu=1,mem=4000M,node=1  2-00:00:00   vmp1242   
8       1     2Gc     NaN     cpu=1,mem=2G,node=1    12:00:00    vmp670   
9       1     2Gc     NaN     cpu=1,mem=2G,node=1    12:00:00   vmp1078   

                                             JobName  
0  WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...  
1  WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...  
2  WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...  
3  WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...  
4                                    bl_aa7c22e04f21  
5                                          skim_test  
6                                          skim_test  
7                                    bl_33ee450ebdc4  
8                                          skim_test  
9                                          skim_test  

[10 rows x 25 columns]

In [4]:
accre_df.tail(10)

JobID  JobIDRaw Cluster   Partition      Account        Group  \
12499379  28331346  28331346   accre  production   cms_stage3          cms   
12499380  28331347  28331347   accre  production   cms_stage3          cms   
12499381  28331348  28331348   accre  production   cms_stage3          cms   
12499382  28331349  28331349   accre  production   cms_stage3          cms   
12499383  28331350  28331350   accre  production   cms_stage3          cms   
12499384  28331351  28331351   accre  production   cms_stage3          cms   
12499385  28331352  28331352   accre  production   cms_stage3          cms   
12499386  28331353  28331353   accre  production   cms_stage3          cms   
12499387  28331356  28331356   accre  production   cms_stage2          cms   
12499388  28331357  28331357   accre  production  cms_samtest  cms_samtest   

            GID      User       UID               Submit         ...           \
12499379  31000  janjamrk  171357.0  2018-06-30T23:54:17         ...            
12499380  31000  janjamrk  171357.0  2018-06-30T23:54:17         ...            
12499381  31000  janjamrk  171357.0  2018-06-30T23:54:18         ...            
12499382  31000  janjamrk  171357.0  2018-06-30T23:54:18         ...            
12499383  31000  janjamrk  171357.0  2018-06-30T23:54:20         ...            
12499384  31000  janjamrk  171357.0  2018-06-30T23:54:21         ...            
12499385  31000  janjamrk  171357.0  2018-06-30T23:54:22         ...            
12499386  31000  janjamrk  171357.0  2018-06-30T23:54:22         ...            
12499387  31000   autocms    9209.0  2018-06-30T23:55:01         ...            
12499388  59297  uscms010       NaN  2018-06-30T23:55:31         ...            

              State NNodes NCPUS ReqCPUS  ReqMem ReqGRES  \
12499379  COMPLETED      1     1       1     4Gc     NaN   
12499380  COMPLETED      1     1       1     4Gc     NaN   
12499381  COMPLETED      1     1       1     4Gc     NaN   
12499382     FAILED      1     1       1     4Gc     NaN   
12499383     FAILED      1     1       1     4Gc     NaN   
12499384     FAILED      1     1       1     4Gc     NaN   
12499385     FAILED      1     1       1     4Gc     NaN   
12499386     FAILED      1     1       1     4Gc     NaN   
12499387  COMPLETED      1     1       1     2Gc     NaN   
12499388  COMPLETED      1     1       1  4000Mn     NaN   

                         ReqTRES   Timelimit  NodeList              JobName  
12499379     cpu=1,mem=4G,node=1    01:00:00   vmp1331  2018-06-30-040817_1  
12499380     cpu=1,mem=4G,node=1    01:00:00   vmp1336  2018-06-30-040817_2  
12499381     cpu=1,mem=4G,node=1    01:00:00   vmp1347  2018-06-30-040817_3  
12499382     cpu=1,mem=4G,node=1    01:00:00   vmp1385  2018-06-30-040817_4  
12499383     cpu=1,mem=4G,node=1    01:00:00   vmp1406  2018-06-30-040817_5  
12499384     cpu=1,mem=4G,node=1    01:00:00   vmp1314  2018-06-30-040817_6  
12499385     cpu=1,mem=4G,node=1    01:00:00   vmp1315  2018-06-30-040817_7  
12499386     cpu=1,mem=4G,node=1    01:00:00   vmp1315  2018-06-30-040817_8  
12499387     cpu=1,mem=2G,node=1    12:00:00   vmp1065            skim_test  
12499388  cpu=1,mem=4000M,node=1  2-00:00:00   vmp1298      bl_0e20773d4484  

[10 rows x 25 columns]

In [5]:
print(accre_df.columns)
print(accre_df.shape)

Index(['JobID', 'JobIDRaw', 'Cluster', 'Partition', 'Account', 'Group', 'GID',
       'User', 'UID', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed',
       'ExitCode', 'State', 'NNodes', 'NCPUS', 'ReqCPUS', 'ReqMem', 'ReqGRES',
       'ReqTRES', 'Timelimit', 'NodeList', 'JobName'],
      dtype='object')
(12499389, 25)


## Queue time = Start time - submit time

Queue time is the amount of time that a user has to wait until the job runs

## We want to see how requested RAM (reqMem) and requested wall time (Timelimit) affect the Queue time

In [6]:
# Checking if elapsed time is the same as difference between start and end
print(accre_df[['Start', 'End', 'Elapsed']].head(30))

                  Start                  End     Elapsed
0   2017-07-03T02:13:59  2017-07-04T03:19:20  1-01:05:21
1   2017-07-03T02:20:08  2017-07-04T02:11:55    23:51:47
2   2017-07-03T02:41:13  2017-07-04T04:41:04  1-01:59:51
3   2017-07-03T02:51:53  2017-07-04T10:54:13  1-08:02:20
4   2017-07-01T05:00:01  2017-07-01T05:00:11    00:00:10
5   2017-07-01T05:00:04  2017-07-01T05:03:22    00:03:18
6   2017-07-01T05:00:04  2017-07-01T05:02:07    00:02:03
7   2017-07-01T05:00:23  2017-07-01T05:00:33    00:00:10
8   2017-07-01T05:01:02  2017-07-01T05:05:49    00:04:47
9   2017-07-01T05:01:05  2017-07-01T05:04:17    00:03:12
10  2017-07-01T05:03:05  2017-07-05T15:03:31  4-10:00:26
11  2017-07-01T05:24:23  2017-07-04T23:31:41  3-18:07:18
12  2017-07-01T05:38:05  2017-07-05T08:57:48  4-03:19:43
13  2017-07-01T06:03:05  2017-07-05T15:47:51  4-09:44:46
14  2017-07-01T06:26:59  2017-07-05T09:24:54  4-02:57:55
15  2017-07-01T06:41:30  2017-07-06T02:26:56  4-19:45:26
16  2017-07-01T07:03:05  2017-0

In [7]:
# Get the data types and size of each of the columns
accre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12499389 entries, 0 to 12499388
Data columns (total 25 columns):
JobID        object
JobIDRaw     int64
Cluster      object
Partition    object
Account      object
Group        object
GID          int64
User         object
UID          float64
Submit       object
Eligible     object
Start        object
End          object
Elapsed      object
ExitCode     object
State        object
NNodes       int64
NCPUS        int64
ReqCPUS      int64
ReqMem       object
ReqGRES      object
ReqTRES      object
Timelimit    object
NodeList     object
JobName      object
dtypes: float64(1), int64(5), object(19)
memory usage: 2.3+ GB


## Submit and start times are both strings, so we need to convert those to a form that we can use

In [8]:
# Checking the components of the Eligible column
accre_df.Eligible.head()


0    2017-07-02T00:42:59
1    2017-07-02T00:42:59
2    2017-07-02T00:42:59
3    2017-07-02T00:42:59
4    2017-07-01T05:00:00
Name: Eligible, dtype: object

In [9]:
# Checking the components of the Submit column
accre_df.Submit.head()

0    2017-07-02T00:40:58
1    2017-07-02T00:40:58
2    2017-07-02T00:40:58
3    2017-07-02T00:40:58
4    2017-07-01T05:00:00
Name: Submit, dtype: object

## We want to get the Timelimit column in a standard format

Currently the format is d-h:m:s
However, this format breaks down if the requested time is less than 1 day. This makes the format h:m:s
Need to standardize the format so that it can be changed to datetime format

In [10]:
print(type(accre_df['Timelimit'].iloc[5]))

<class 'str'>


In [134]:
# Add a 0- in front of those that are less than a day
# create a mask of indexes where the 3rd character is ':'
# Then add '0-' in front of those strings in the Timelimit column
dmask = accre_df['Timelimit'].str[2] == ':'
#accre_df.loc[dmask, 'Timelimit'] = '0-' + accre_df['Timelimit']
print(accre_df['Timelimit'].head(30))

0     9-04:00:00
1     9-04:00:00
2     9-04:00:00
3     9-04:00:00
4     2-00:00:00
5       12:00:00
6       12:00:00
7     2-00:00:00
8       12:00:00
9       12:00:00
10    5-08:00:00
11    5-08:00:00
12    5-08:00:00
13    5-08:00:00
14    5-08:00:00
15    5-08:00:00
16    5-08:00:00
17    5-08:00:00
18    5-08:00:00
19    5-08:00:00
20    5-08:00:00
21    5-08:00:00
22    5-08:00:00
23    5-08:00:00
24    5-08:00:00
25    5-08:00:00
26    5-08:00:00
27    5-08:00:00
28    5-08:00:00
29    5-08:00:00
Name: Timelimit, dtype: object


In [67]:
print(type(accre_df['Timelimit'].iloc[3]))

<class 'str'>


In [143]:
# Change the submit and start column to a date time 64 data type
accre_df['Submit'] = pd.to_datetime(accre_df['Submit'], format="%Y/%m/%d")
accre_df['Start'] = pd.to_datetime(accre_df['Start'], format="%Y/%m/%d")
#Convert timelimit as well because we'll need it later
# accre_df['Timelimit'] = pd.to_datetime(accre_df['Timelimit'], format = "%H:%M:%S", exact = True)
# test_td = pd.to_datetime(accre_df['Timelimit'], infer_datetime_format=True)   #, format = "%d-%H:%M:%S", exact = True)

In [144]:
accre_df.Start.head()

0   2017-07-03 02:13:59
1   2017-07-03 02:20:08
2   2017-07-03 02:41:13
3   2017-07-03 02:51:53
4   2017-07-01 05:00:01
Name: Start, dtype: datetime64[ns]

## Subtract the submit time from the queue time to find the queue time

In [145]:
accre_df['queue_time'] = accre_df['Start'] - accre_df['Submit']
print(accre_df.queue_time.head(30))

0    1 days 01:33:01
1    1 days 01:39:10
2    1 days 02:00:15
3    1 days 02:10:55
4    0 days 00:00:01
5    0 days 00:00:03
6    0 days 00:00:02
7    0 days 00:00:01
8    0 days 00:00:01
9    0 days 00:00:03
10   0 days 00:01:43
11   0 days 00:23:01
12   0 days 00:36:42
13   0 days 01:01:42
14   0 days 01:25:36
15   0 days 01:40:07
16   0 days 02:01:42
17   0 days 03:01:45
18   0 days 03:41:41
19   0 days 03:51:59
20   0 days 04:00:01
21   0 days 05:30:14
22   0 days 05:36:41
23   0 days 05:46:40
24   0 days 05:58:32
25   0 days 07:26:40
26   0 days 07:35:36
27   0 days 07:47:14
28   0 days 08:26:39
29   0 days 08:46:39
Name: queue_time, dtype: timedelta64[ns]


In [146]:
accre_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12499389 entries, 0 to 12499388
Data columns (total 26 columns):
JobID         object
JobIDRaw      int64
Cluster       object
Partition     object
Account       object
Group         object
GID           int64
User          object
UID           float64
Submit        datetime64[ns]
Eligible      object
Start         datetime64[ns]
End           object
Elapsed       object
ExitCode      object
State         object
NNodes        int64
NCPUS         int64
ReqCPUS       int64
ReqMem        object
ReqGRES       object
ReqTRES       object
Timelimit     object
NodeList      object
JobName       object
queue_time    timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(17), timedelta64[ns](1)
memory usage: 2.4+ GB


## Convert the timedelta64 from the queue_time column to seconds (float) for further processing

In [147]:
# Convert the queue time to a floating point number containing the seconds it took to start
accre_df['queue_time'] = accre_df.queue_time.dt.total_seconds()
print(accre_df['queue_time'].head())

0    91981.0
1    92350.0
2    93615.0
3    94255.0
4        1.0
Name: queue_time, dtype: float64


## Now we have the queue time in a floating point number
## Next we need to get the requested memory and requested wall time in a usable format

## We also need to get the Requested Time ('Timelimit') in a usable format

In [156]:
# Get Days, hours, minutes, and seconds
req_days = pd.to_numeric(accre_df['Timelimit'].str[:-9]).fillna(value=0)
req_hours = pd.to_numeric(accre_df['Timelimit'].str[-8:-6])
req_minutes = pd.to_numeric(accre_df['Timelimit'].str[-5:-3])
req_secs = pd.to_numeric(accre_df['Timelimit'].str[-2:])
# Apply formula to get total time in seconds
accre_df['req_time'] = (req_days*86400)+(req_hours*3600)+(req_minutes*60)+req_secs

print(accre_df['req_time'].head(20))

0     792000.0
1     792000.0
2     792000.0
3     792000.0
4     172800.0
5      43200.0
6      43200.0
7     172800.0
8      43200.0
9      43200.0
10    460800.0
11    460800.0
12    460800.0
13    460800.0
14    460800.0
15    460800.0
16    460800.0
17    460800.0
18    460800.0
19    460800.0
Name: req_time, dtype: float64


## Now we have the requested wall time in a column called req_time

In [148]:
# Requested memory is in the column ReqMem
# Number at beginning signifies amount of RAM in either MB or GB, depending if the letter is M or G
# The amount of RAM is per core or node, depending whether the last letter is c or n
gig_ind_percore = accre_df['ReqMem'].str[-2:] == 'Gc'
meg_ind_percore = accre_df['ReqMem'].str[-2:] == 'Mc'
gig_ind_pernode = accre_df['ReqMem'].str[-2:] == 'Gn'
meg_ind_pernode = accre_df['ReqMem'].str[-2:] == 'Mn'

print(gig_ind_percore.tail(20))


12499369     True
12499370     True
12499371     True
12499372     True
12499373     True
12499374     True
12499375     True
12499376     True
12499377     True
12499378     True
12499379     True
12499380     True
12499381     True
12499382     True
12499383     True
12499384     True
12499385     True
12499386     True
12499387     True
12499388    False
Name: ReqMem, dtype: bool


## Convert the requested memory to floating point values

In [158]:
# Taking everything up to the second to last character because that will be the amount of memory requested
accre_df['mem_int'] = accre_df['ReqMem'].str[:-2]
accre_df['mem_int'] = pd.to_numeric(accre_df.mem_int)
print(accre_df.mem_int.head(20))

0       12.0
1       12.0
2       12.0
3       12.0
4     4000.0
5        2.0
6        2.0
7     4000.0
8        2.0
9        2.0
10      10.0
11      10.0
12      10.0
13      10.0
14      10.0
15      10.0
16      10.0
17      10.0
18      10.0
19      10.0
Name: mem_int, dtype: float64


## Convert all of the requested memory to GB

This makes it a little easier to handle if everything is in the same unit

In [159]:
# Divide the "Mc" values by 1000 to get to GB
accre_df.loc[meg_ind_percore, 'mem_int'] = accre_df['mem_int'] / 1000
# Divide the "Mn" values by 1000 to get to GB
accre_df.loc[meg_ind_pernode, 'mem_int'] = accre_df['mem_int'] / 1000
print(accre_df.mem_int.head(20))

0     12.0
1     12.0
2     12.0
3     12.0
4      4.0
5      2.0
6      2.0
7      4.0
8      2.0
9      2.0
10    10.0
11    10.0
12    10.0
13    10.0
14    10.0
15    10.0
16    10.0
17    10.0
18    10.0
19    10.0
Name: mem_int, dtype: float64


In [160]:
print(accre_df.mem_int.sort_values(ascending = False).head(30))

8673264     100000.0
5062374       2000.0
5062383       2000.0
5062390       2000.0
5062375       2000.0
5062376       2000.0
5062377       2000.0
5062378       2000.0
5062379       2000.0
5062380       2000.0
5062381       2000.0
5062382       2000.0
5062384       2000.0
5062372       2000.0
5062385       2000.0
5062386       2000.0
5062387       2000.0
5062388       2000.0
5062389       2000.0
5062394       2000.0
5062393       2000.0
5062392       2000.0
5062373       2000.0
5062391       2000.0
5062371       2000.0
5062369       2000.0
5062370       2000.0
11916731      1000.0
8675470       1000.0
8675469       1000.0
Name: mem_int, dtype: float64


In [161]:
# Creating a new column with n or c to signify if the requested memory is per core or per node
accre_df['core_node'] = accre_df['ReqMem'].str[-1]
print(accre_df.core_node.head(20))

0     n
1     n
2     n
3     n
4     n
5     c
6     c
7     n
8     c
9     c
10    n
11    n
12    n
13    n
14    n
15    n
16    n
17    n
18    n
19    n
Name: core_node, dtype: object


In [162]:
print(accre_df.iloc[11916731,:])

JobID                       27745464
JobIDRaw                    27745464
Cluster                        accre
Partition                 production
Account                   bowman_lab
Group                      capra_lab
GID                            20429
User                          yangkl
UID                           348967
Submit           2018-06-04 06:38:30
Eligible         2018-06-04T06:38:30
Start            2018-06-04 06:38:30
End              2018-06-04T06:38:30
Elapsed                     00:00:00
ExitCode                         1:0
State                         FAILED
NNodes                             1
NCPUS                              1
ReqCPUS                            1
ReqMem                        1000Gn
ReqGRES                          NaN
ReqTRES       cpu=1,mem=1000G,node=1
Timelimit                   06:00:00
NodeList               None assigned
JobName                 compress.txt
queue_time                         0
req_time                       21600
m